In [ ]:
pip install nselib
pip install schedule


Note: you may need to restart the kernel to use updated packages.


In [4]:
import nselib
from nselib import capital_market
import pandas as pd


5665 days, 0:00:00


In [ ]:
import os
from datetime import datetime, timedelta, date
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
import holidays
from nselib import capital_market

# Setup
output_file = "bhavcopy_data.csv"
in_holidays = holidays.India()
business_days = CustomBusinessDay(holidays=in_holidays)

# Date range
first_date=date(2020,1,1)
today = date.today()
diff= (today - first_date).days


now = datetime.now()
end = datetime.today() if now > now.replace(hour=9, minute=20) else datetime.today() - timedelta(days=1)
start = end - timedelta(days=diff)
dates = pd.date_range(start, end, freq=business_days)[-diff:]
date_strs = [d.strftime('%d-%m-%Y') for d in (dates)]

# Existing dates
existing = set()
if os.path.exists(output_file):
    existing = set(pd.read_csv(output_file, usecols=["date"])["date"].unique())

# Fetch and append new data
new_data = []
for ds in date_strs:
    if ds in existing:
        print(f"Skipping {ds}")
        continue
    try:
        df = capital_market.bhav_copy_with_delivery(ds)
        df["date"] = ds
        new_data.append(df)
        print(f"Fetched {ds}")
    except Exception as e:
        print(f"Failed {ds}: {e}")

if new_data:
    combined = pd.concat(new_data)
    combined.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
    print("Data saved.")
else:
    print("No new data.")


Fetched 01-01-2020
Fetched 02-01-2020
Fetched 03-01-2020
Fetched 06-01-2020
Fetched 07-01-2020
Fetched 08-01-2020
